In [1]:
import pandas as pd
from util.performance_metrics import *
 
url =  "https://raw.githubusercontent.com/xuda1979/DF/master/sp500_companies/data/constituents.csv"
df_symbols =pd.read_csv('./sp500_companies/data/constituents.csv')

In [15]:
df_symbols.head()
all_symbols = list(df_symbols['Symbol'].unique())
all_symbols.append('SPY')

In [16]:
# Import yfinance
import yfinance as yf  
 
# Get the data for the stock Apple by specifying the stock ticker, start date, and end date
data_all =pd.DataFrame({})
for symbol in all_symbols:
    
    data = yf.download(symbol)#,'2016-01-01','2018-01-01')
    data['Symbol'] = symbol
    if(len(data)>0):
        data_all = pd.concat([data_all,data])
 

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- HRS: No data found, symbol may be delisted
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- HCP: No data found, symbol may be delisted
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%****************

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- TMK: No data found, symbol may be delisted
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- TSS: No data found, symbol may be delisted
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%****************

In [20]:
symbols = list(data_all['Symbol'].unique())

data_all.reset_index(inplace=True)

data_all.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Symbol
0,1970-01-02,6.851562,6.890625,6.843750,6.851562,1.483333,72000.0,MMM
1,1970-01-05,6.859375,6.898438,6.859375,6.890625,1.491789,446400.0,MMM
2,1970-01-06,6.890625,6.960938,6.882812,6.960938,1.507011,176000.0,MMM
3,1970-01-07,6.960938,7.015625,6.945312,7.000000,1.515469,164800.0,MMM
4,1970-01-08,7.000000,7.109375,6.984375,7.093750,1.535765,304000.0,MMM


In [28]:
training_start_date = '2015-01-15'
training_end_date = '2020-09-30'
mask = data_all['Date'] >= training_start_date
mask &= data_all['Date'] <= training_end_date

data_training= data_all[mask] 

In [10]:
def date_filter(df, start_date, end_date):
    mask = df['Date'] >=  start_date
    mask &= df['Date'] <= end_date
    return df[mask]

# Risk Model

In [931]:
def covariance(df, date, symbols, history):
    df_recent = pd.DataFrame({})
    dates = list(df['Date'].unique())
    dates.sort()
    df_recent['Date']= dates[-(history+1):-1]
    for symbol in symbols:
        mask = df['Symbol'] == symbol
 
        if all([date in list(df[mask]['Date'].unique()) for date in dates[-(history+1):-1]]):
            #print(df[mask]['Close'][-(history+1):-1])
            df_recent[symbol] = df[mask]['Close'].to_list()[-(history+1):-1]
    df_return = pd.DataFrame({})
    df_return['Date'] = df_recent['Date']
    for symbol in df_recent.columns:
        if symbol !='Date':
            #print(symbol,df_recent[symbol].div(df_recent[symbol].shift(1)))
            df_return[symbol] = df_recent[symbol].div(df_recent[symbol].shift(1))
    return df_return[1:].set_index('Date').cov() 
 
            
symbols = ['AAPL', 'MMM']
df_cov = covariance(data_training, list(data_training['Date'].unique())[-1], symbols, 10)
cov=df_cov.to_numpy() 
print(cov)

[[7.85042934e-04 7.44584306e-06]
 [7.44584306e-06 3.88371563e-04]]


In [988]:
def objective(x):
    l = len(x)
    assert l == cov.shape[0]
    return x.dot(cov.dot(x))

def constraint(x):
    return np.atleast_1d(np.sum(np.abs(x)) - 1.5)

 
bounds=[[0, 1], [1, 2]]
res=optimize.minimize(objective, np.array([0, 0]), constraints={"fun": constraint, "type": "ineq"},options={'gtol': 1e-8, 'disp': True}, bounds=bounds )
res.x    

Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.0006603349124938092
            Iterations: 2
            Function evaluations: 6
            Gradient evaluations: 2


<ipython-input-988-e88c3116b488>:11: OptimizeWarning: Unknown solver options: gtol
  res=optimize.minimize(objective, np.array([0, 0]), constraints={"fun": constraint, "type": "ineq"},options={'gtol': 1e-8, 'disp': True}, bounds=bounds )


array([0.25038093, 1.24961907])

In [1025]:
from scipy.optimize import minimize
def portfolio_optimization(cov, k):        
 
    assert 2*k == cov.shape[0]
    def objective(x):
        return x.dot(cov.dot(x))

    def constraint(x):
        return np.atleast_1d(np.sum(np.abs(x))-1)
    x0 = [1/(2*k)]*k+[-1/(2*k)]*k
    x0 = np.array(x0)
    bounds = []
    for i in range(k):
        bounds.append([1/(4*k),1/k])
    for i in range(k):
        bounds.append([-1/k, -1/(4*k)])
        
    res=optimize.minimize(objective, x0, constraints={"fun": constraint, "type": "ineq"},options={'gtol': 1e-8, 'disp': False}, bounds=bounds )
    s= sum(np.abs(res.x))
    return res.x/s 

# Factor model  

#### Genearte DataFrames of various returns for each symbol 

In [29]:
data_training_return = {}
 
for symbol in symbols:
    print(symbol)
    mask = data_training['Symbol'] == symbol
    data_training_return[symbol] = pd.DataFrame({})
    data_training_return[symbol]['Date'] = data_training[mask]['Date']
    
    data_training_return[symbol]['close_close_return']  = data_training[mask]['Close'].div(data_training[mask]['Close'].shift(1))-1  
    data_training_return[symbol]['open_close_return']  = data_training[mask]['Close'].div(data_training[mask]['Open'] )-1
    data_training_return[symbol]['close_open_return']  = data_training[mask]['Open'].div(data_training[mask]['Close'].shift(1) )-1
    data_training_return[symbol]['vwap_open_return']  = 4*data_training[mask]['Open'].div(data_training[mask]['Open'].shift(1)+ \
                                                                                        data_training[mask]['High'].shift(1)+ \
                                                                                        data_training[mask]['Low'].shift(1)+ \
                                                                                        data_training[mask]['Close'].shift(1))-1
    
    data_training_return[symbol]['vwap_close_return']  = 4*data_training[mask]['Open'].div(data_training[mask]['Open']+ \
                                                                                        data_training[mask]['High']+ \
                                                                                        data_training[mask]['Low']+ \
                                                                                        data_training[mask]['Close'])-1
    
    
    data_training_return[symbol].fillna(0, inplace=True)
    data_training_return[symbol].reset_index(inplace=True,drop=True)
    
   


MMM
AOS
ABT
ABBV
ACN
ATVI
AYI
ADBE
AAP
AMD
AES
AET
AMG
AFL
A
APD
AKAM
ALK
ALB
ARE
ALXN
ALGN
ALLE
ADS
LNT
ALL
GOOGL
GOOG
MO
AMZN
AEE
AAL
AEP
AXP
AIG
AMT
AWK
AMP
ABC
AME
AMGN
APH
ADI
ANDV
ANSS
ANTM
AON
APA
AIV
AAPL
AMAT
APTV
ADM
ARNC
AJG
AIZ
T
ADSK
ADP
AZO
AVB
AVY
BLL
BAC
BAX
BDX
BBY
BIIB
BLK
HRB
BA
BWA
BXP
BSX
BHF
BMY
AVGO
CHRW
CA
COG
CDNS
CPB
COF
CAH
KMX
CCL
CAT
CBOE
CNC
CNP
CTL
CERN
CF
SCHW
CHTR
CHK
CVX
CMG
CB
CHD
CI
XEC
CINF
CTAS
CSCO
C
CFG
CTXS
CME
CMS
KO
CTSH
CL
CMCSA
CMA
CAG
CXO
COP
ED
STZ
GLW
COST
COTY
CCI
CSRA
CSX
CMI
CVS
DHI
DHR
DRI
DVA
DE
DAL
XRAY
DVN
DLR
DFS
DISCA
DISCK
DISH
DG
DLTR
D
DOV
DTE
DUK
DRE
DXC
ETFC
EMN
ETN
EBAY
ECL
EIX
EW
EA
EMR
ETR
EVHC
EOG
EQT
EFX
EQIX
EQR
ESS
EL
RE
ES
EXC
EXPE
EXPD
ESRX
EXR
XOM
FFIV
FB
FAST
FRT
FDX
FIS
FITB
FE
FISV
FLIR
FLS
FLR
FMC
FL
F
FTV
FBHS
BEN
FCX
GPS
GRMN
IT
GD
GE
GIS
GM
GPC
GILD
GPN
GS
GT
GWW
HAL
HBI
HOG
HIG
HAS
HCA
HP
HSIC
HES
HPE
HLT
HOLX
HD
HON
HRL
HST
HPQ
HUM
HBAN
HII
IDXX
INFO
ITW
ILMN
INCY
IR
INTC
ICE
IBM
IP
IPG
IFF
INTU
ISRG
IVZ
I

#### Train the regression model 

In [71]:
testing_length = 250
training_dates= set(data_training_return['SPY']['Date'][:-testing_length])


import numpy as np
from sklearn.linear_model import LinearRegression
X = data_training_return['SPY']['close_close_return'] 
dates_X = set(data_training_return['SPY']['Date'] )
dates_X = dates_X.intersection(training_dates)

reg_map = {}
for symbol in symbols:
 
 
    
        dates_y = set(data_training_return[symbol]['Date'] )
        dates = dates_y.intersection(dates_X)
        mask = data_training_return[symbol]['Date'].isin(dates)
        y =  data_training_return[symbol][mask]['close_close_return'] 

        mask = data_training_return['SPY']['Date'].isin(dates)
        X_here = data_training_return['SPY'][mask]['close_close_return'] 
        X_here = np.expand_dims(X_here, axis=1)
        print(X_here.shape, y.shape, symbol)
        if X_here.shape[0] == len(dates_X):
            reg = LinearRegression().fit(X_here, y)
            reg_map[symbol] = reg
 
# reg.score(X, y)

# reg.coef_

# reg.intercept_
 
 

(1188, 1) (1188,) MMM
(1188, 1) (1188,) AOS
(1188, 1) (1188,) ABT
(1188, 1) (1188,) ABBV
(1188, 1) (1188,) ACN
(1188, 1) (1188,) ATVI
(1188, 1) (1188,) AYI
(1188, 1) (1188,) ADBE
(1188, 1) (1188,) AAP
(1188, 1) (1188,) AMD
(1188, 1) (1188,) AES
(977, 1) (977,) AET
(1188, 1) (1188,) AMG
(1188, 1) (1188,) AFL
(1188, 1) (1188,) A
(1188, 1) (1188,) APD
(1188, 1) (1188,) AKAM
(1188, 1) (1188,) ALK
(1188, 1) (1188,) ALB
(1188, 1) (1188,) ARE
(1188, 1) (1188,) ALXN
(1188, 1) (1188,) ALGN
(1188, 1) (1188,) ALLE
(1188, 1) (1188,) ADS
(1188, 1) (1188,) LNT
(1188, 1) (1188,) ALL
(1188, 1) (1188,) GOOGL
(1188, 1) (1188,) GOOG
(1188, 1) (1188,) MO
(1188, 1) (1188,) AMZN
(1188, 1) (1188,) AEE
(1188, 1) (1188,) AAL
(1188, 1) (1188,) AEP
(1188, 1) (1188,) AXP
(1188, 1) (1188,) AIG
(1188, 1) (1188,) AMT
(1188, 1) (1188,) AWK
(1188, 1) (1188,) AMP
(1188, 1) (1188,) ABC
(1188, 1) (1188,) AME
(1188, 1) (1188,) AMGN
(1188, 1) (1188,) APH
(1188, 1) (1188,) ADI
(934, 1) (934,) ANDV
(1188, 1) (1188,) ANSS
(11

(1188, 1) (1188,) DIS
(1188, 1) (1188,) TMO
(1188, 1) (1188,) TIF
(860, 1) (860,) TWX
(1188, 1) (1188,) TJX
(1188, 1) (1188,) TSCO
(1188, 1) (1188,) TDG
(1188, 1) (1188,) TRIP
(144, 1) (144,) FOXA
(143, 1) (143,) FOX
(1188, 1) (1188,) TSN
(1188, 1) (1188,) USB
(1188, 1) (1188,) UDR
(1188, 1) (1188,) ULTA
(1188, 1) (1188,) UAA
(1068, 1) (1068,) UA
(1188, 1) (1188,) UNP
(1188, 1) (1188,) UAL
(1188, 1) (1188,) UNH
(1188, 1) (1188,) UPS
(1188, 1) (1188,) URI
(1188, 1) (1188,) UHS
(1188, 1) (1188,) UNM
(1188, 1) (1188,) VFC
(1188, 1) (1188,) VLO
(1188, 1) (1188,) VAR
(1188, 1) (1188,) VTR
(1188, 1) (1188,) VRSN
(1188, 1) (1188,) VRSK
(1188, 1) (1188,) VZ
(1188, 1) (1188,) VRTX
(1188, 1) (1188,) V
(1188, 1) (1188,) VNO
(1188, 1) (1188,) VMC
(1188, 1) (1188,) WMT
(1188, 1) (1188,) WBA
(1188, 1) (1188,) WM
(1188, 1) (1188,) WAT
(1188, 1) (1188,) WEC
(1188, 1) (1188,) WFC
(1188, 1) (1188,) WDC
(1188, 1) (1188,) WU
(1078, 1) (1078,) WRK
(1188, 1) (1188,) WY
(1188, 1) (1188,) WHR
(1188, 1) (1188,

#### Trade at open 

In [97]:

c= 0.0003

result = np.zeros(250)
k= 2

testing_dates = list(data_training_return['SPY']['Date'][-testing_length:])
for index, date in enumerate(testing_dates):
    error_map = {}
    for symbol in symbols:
        
        if symbol in reg_map and symbol !='SPY':
   

 
            try:
                mask = data_training_return['SPY']['Date'] == date
                X = data_training_return['SPY'][mask]['close_open_return'].to_numpy()
                X = np.expand_dims(X, axis=1)
                pred = reg_map[symbol].predict(X)
                mask = data_training_return[symbol]['Date'] == date
                error = data_training_return[symbol][mask]['close_open_return'].to_numpy()[0] - pred[0]
                error_map[error] = symbol
            except:
                pass
    error_list = list(error_map.items())
    error_list.sort()
 
 
    for j in range(k):
        if j< len(error_list):   
            symbol = error_list[j][1]
            mask = data_training_return[symbol]['Date'] == date
            result[index] += (1/(2*k))*data_training_return[symbol][mask]['open_close_return'].to_numpy()[0] 
            
            symbol = error_list[-j-1][1]
            mask = data_training_return[symbol]['Date'] == date
            result[index] -= (1/(2*k))*data_training_return[symbol][mask]['open_close_return'].to_numpy()[0] 

Performance(result, c)
            

Daily mean return:  0.004011713428848835
Sharp ratio:  2.1599123311370256


In [26]:
result = np.zeros(testing_length)
k= 5

testing_dates = list(data_training_return['SPY']['Date'][-testing_length:])
for index, date in enumerate(testing_dates):
    error_map = {}
    for symbol in symbols:
        
        if symbol in reg_map and symbol !='SPY':
   

 
            try:
                mask = data_training_return['SPY']['Date'] == date
                X = data_training_return['SPY'][mask]['vwap_open_return'].to_numpy()
                X = np.expand_dims(X, axis=1)
                pred = reg_map[symbol].predict(X)
                mask = data_training_return[symbol]['Date'] == date
                error = data_training_return[symbol][mask]['vwap_open_return'].to_numpy()[0] - pred[0]
                error_map[error] = symbol
            except:
                pass
    error_list = list(error_map.items())
    error_list.sort()
 
 
    for j in range(k):
        if j< len(error_list):   
            symbol = error_list[j][1]
            mask = data_training_return[symbol]['Date'] == date
            result[index] += (1/(2*k))*data_training_return[symbol][mask]['open_close_return'].to_numpy()[0] 
            
            symbol = error_list[-j-1][1]
            mask = data_training_return[symbol]['Date'] == date
            result[index] -= (1/(2*k))*data_training_return[symbol][mask]['open_close_return'].to_numpy()[0] 

Performance(result, c)
            

Daily mean return:  -1999.997420737242
Sharp ratio:  -1607892.691646151


#### Trade at close 

In [27]:
result = np.zeros(250)
k=5
testing_dates = list(data_training_return['SPY']['Date'][-250:])
for index, date in enumerate(testing_dates):
    error_map = {}
    for symbol in symbols:
        
        if symbol in reg_map and symbol !='SPY':
   

 
            try:
                mask = data_training_return['SPY']['Date'] == date
                X = data_training_return['SPY'][mask]['vwap_close_return']
                X = np.expand_dims(X, axis=1)
                pred = reg_map[symbol].predict(X)
                mask = data_training_return[symbol]['Date'] == date
                error = data_training_return[symbol][mask]['vwap_close_return'].to_numpy()[0] - pred[0]
                error_map[error] = symbol
            except:
                pass
    error_list = list(error_map.items())
    error_list.sort()
 
 
    for j in range(k):
        if j< len(error_list) and index+1<len(testing_dates): 
            try:
                symbol = error_list[j][1]
                mask = data_training_return[symbol]['Date'] == testing_dates[index+1]
                result[index+1] += (1/(2*k))*data_training_return[symbol][mask]['close_open_return'].to_numpy()[0] 

                symbol = error_list[-j-1][1]
                mask = data_training_return[symbol]['Date'] == testing_dates[index+1]
                result[index+1] -= (1/(2*k))*data_training_return[symbol][mask]['close_open_return'].to_numpy()[0] 
            except:
                pass

Performance(result,c)
            

Daily mean return:  -1999.9971705570908
Sharp ratio:  -2014697.521599924


# Mean Reverse Effect at opening

In [84]:
testing_length = 250
training_dates= set(data_training_return['SPY']['Date'][:-testing_length])


import numpy as np
from sklearn.linear_model import LinearRegression
X_open_close = data_training_return['SPY']['open_close_return'] 
X_close_open = data_training_return['SPY']['open_close_return'] 
dates_X = set(data_training_return['SPY']['Date'] )
dates_X = dates_X.intersection(training_dates)

reg_map_reverse = {}
for symbol in symbols:
 
 
    
        dates_y = set(data_training_return[symbol]['Date'] )
        dates = dates_y.intersection(dates_X)
        mask = data_training_return[symbol]['Date'].isin(dates)
        y_open_close =  data_training_return[symbol][mask]['open_close_return']
#         y_open_close = np.expand_dims(y_open_close, axis=1)
        y_close_open = data_training_return[symbol][mask]['close_open_return']
        y_close_open = np.expand_dims(y_close_open, axis=1)

        mask = data_training_return['SPY']['Date'].isin(dates)
        X_open_close = data_training_return['SPY'][mask]['open_close_return'] 
        X_open_close = np.expand_dims(X_open_close, axis=1)
        
        X_close_open = data_training_return['SPY'][mask]['close_open_return'] 
        X_close_open = np.expand_dims(X_close_open, axis=1)
        
        if symbol in reg_map:
            pred_close_open = reg_map[symbol].predict(X_close_open)
            pred_close_open = np.expand_dims(pred_close_open, axis=1)
            error_close_open = y_close_open-pred_close_open
#             print(error_close_open.shape, y_close_open.shape, pred_close_open.shape)
#             break
            X = np.concatenate([X_open_close, error_close_open], axis=1)
 
            if len(dates_X)>0 and X.shape[0] == len(dates_X):
                reg = LinearRegression().fit(X, y_open_close)
                reg_map_reverse[symbol] = reg
 
# reg.score(X, y)

# reg.coef_

# reg.intercept_
 
 

In [96]:
c= 0.0003

result = np.zeros(250)
k= 2

testing_dates = list(data_training_return['SPY']['Date'][-testing_length:])
for index, date in enumerate(testing_dates):
    error_map = {}

    for symbol in symbols:
 
        
        if symbol in reg_map and symbol in reg_map_reverse and symbol !='SPY':
  

   

 
#             try:
 
            mask = data_training_return[symbol]['Date'] == date     
            y_close_open = data_training_return[symbol][mask]['close_open_return']
            y_close_open = np.expand_dims(y_close_open, axis=1)

            mask = data_training_return['SPY']['Date'] == date     
            X_close_open = data_training_return['SPY'][mask]['close_open_return'] 
            X_close_open = np.expand_dims(X_close_open, axis=1)
            
            X_open_close =  np.zeros([len(X_close_open),1])

            if X_open_close.shape == X_close_open.shape == y_close_open.shape:

                pred_close_open = reg_map[symbol].predict(X_close_open)
                pred_close_open = np.expand_dims(pred_close_open, axis=1)
                error_close_open = y_close_open-pred_close_open
                
                X = np.concatenate([X_open_close, error_close_open], axis=1)
#                 print(X.shape)

                pred = reg_map_reverse[symbol].predict(X)

                error_map[pred[0]] = symbol
#             except:
#                 pass
    error_list = list(error_map.items())
    error_list.sort()
 
 
 
    for j in range(k):
        if j< len(error_list):   
            symbol = error_list[j][1]
            mask = data_training_return[symbol]['Date'] == date
            result[index] -= (1/(2*k))*data_training_return[symbol][mask]['open_close_return'].to_numpy()[0] 
            
            symbol = error_list[-j-1][1]
            mask = data_training_return[symbol]['Date'] == date
            result[index] += (1/(2*k))*data_training_return[symbol][mask]['open_close_return'].to_numpy()[0] 

Performance(result, c)
            

Daily mean return:  0.004230407456643464
Sharp ratio:  3.5379229721976664


Random Forests

In [1881]:
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.datasets import make_regression
# X, y = make_regression(n_features=4, n_informative=2,
#                        random_state=0, shuffle=False)
# regr = RandomForestRegressor(n_estimators=1000,max_depth=100, criterion='mae',random_state=0)
# regr.fit(X, y)

# print(regr.predict([[0, 0, 0, 0]]))

[1.63596469]


Generate training samples

In [ ]:
history = 30
testing_length=250
X=[]
y=[]
dates = list(data_training['Date'].unique())
dates.sort()
symbols = set(data_training['Symbol'].unique())
c = 0
for index, date in enumerate(dates[:-testing_length]):
    if c>=1000:
        break
    if index>=history-1:
         
        mask = data_training['Symbol'] == 'SPY'
        mask &= data_training['Date']<date 
        mask &= data_training['Date']>= dates[index-history+1]
                                                    
        sample_spy = list(data_training[mask][['Open','High','Low','Close','Adj Close', 'Volume']].to_numpy().flat)
 
        mask = data_training['Symbol'] == 'SPY'
        mask &= data_training['Date']== date 
    
             
 
        if data_training[mask]['Open'].to_numpy()[0]:
            sample_spy.append(data_training[mask]['Open'].to_numpy()[0])  
            for symbol in symbols:
                c +=1
                if c>=1000:
                    break
                try:
                    sample = []
                    mask = data_training['Symbol'] == symbol
                    mask &= data_training['Date']<date
                    mask &= data_training['Date']>= dates[index-history+1]
                    sample += list(data_training[mask][['Open','High','Low','Close','Adj Close', 'Volume']].to_numpy().flat)
                    sample +=sample_spy



                    mask = data_training_return[symbol]['Date'] == date

                    r = data_training_return[symbol][mask]['close_open_return'].to_numpy()[0]
                    if sample and r:
                        X.append(sample)
                        y.append(r)

                except:
                    print(symbol, date)
    #                 print(data_training_return[symbol]['Date'] )
    #                 raise
X_train=np.array(X)
y_train=np.array(y)
print(X_train.shape, y_train.shape)

In [15]:
print(X)
            

[]


In [13]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
 
regr = RandomForestRegressor(n_estimators=1000,max_depth=100, criterion='mae',random_state=0)
regr.fit(X_train, y_train)

 

ValueError: Expected 2D array, got 1D array instead:
array=[].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [ ]:
history = 30
testing_length=250
k=5
X=[]
y=[]
dates = list(data_training['Date'].unique())
dates.sort()
symbols = set(data_training['Symbol'].unique())
result = np.zeros(testing_length)
print(len(dates))
for index, date in enumerate(dates ):
 
    if index>=len(dates)-testing_length:
         
        mask = data_training['Symbol'] == 'SPY'
        mask &= data_training['Date']<date 
        mask &= data_training['Date']>= dates[index-history+1]
                                                    
        sample_spy = list(data_training[mask][['Open','High','Low','Close','Adj Close', 'Volume']].to_numpy().flat)
 
        mask = data_training['Symbol'] == 'SPY'
        mask &= data_training['Date']== date 
    
        sample_spy.append(data_training[mask]['Open'].to_numpy()[0]) 
        print(sample_spy)
        symbol_return = {}
        symbol_pred_return = {}
        for symbol in symbols:
            try:
                sample = []
                mask = data_training['Symbol'] == symbol
                mask &= data_training['Date']<date
                mask &= data_training['Date']>= dates[index-history+1]
                sample += list(data_training[mask][['Open','High','Low','Close','Adj Close', 'Volume']].to_numpy().flat)
                sample +=sample_spy
                


                mask = data_training_return[symbol]['Date'] == date
         
                r = data_training_return[symbol][mask]['close_open_return'].to_numpy()[0]
                if sample and r:
                    X.append(sample)
                    symbol_pred_return[symbol] = regr.predict(X)
                    symbol_return[symbol] = r 
                
            except:
                print("Error", symbol, date)
#                 print(data_training_return[symbol]['Date'] )
#                 raise
        error_map = {}
        for symbol in  symbol_return:
            error_map[symbol] = symbol_return[symbol] - symbol_pred_return[symbol]
        error_list = list(error_map.items())
        error_list.sort()
       # print(error_list)
        if len(error_list)>=k:
            for j in range(k): 
                symbol = error_list[j][1]
                print(symbol)
                result[index-len(dates)+testing_length] +=(1/(2*k)*data_training[symbol]['open_close_return']
                symbol = error_list[-j-1][1]
                result[index-len(dates)+testing_length] -=(1/(2*k)*data_training[symbol]['open_close_return']
    X=np.array(X)
y=np.array(y)
print(X.shape, y.shape)
Performance(result, c)
            